In [1]:
#IMPORT PACKAGES
import sys
import os
from rdflib import Graph
from pathlib import Path
import rdflib
import open3d as o3d
import numpy as np
import cv2
import ezdxf as cad
import pye57 
import time
import ifcopenshell

import shapely


# from rdflib import Graph, URIRef
# import os.path
# import importlib
# import numpy as np
# import xml.etree.ElementTree as ET
# import open3d as o3d
# import uuid    
# import pye57 
# import ifcopenshell
# import ifcopenshell.geom as geom
# import ifcopenshell.util
# from ifcopenshell.util.selector import Selector
# import random as rd
# import pandas as pd
# from tabulate import tabulate
# import cv2


# #IMPORT MODULES
# from context import geomapi 
# from geomapi.nodes import *
# import geomapi.utils as ut
# from geomapi.utils import geometryutils as gmu
# import geomapi.tools as tl

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


ModuleNotFoundError: No module named 'shapely'

In [ ]:
%autoreload 2

In [ ]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# #GEOMAPI
parent_dir = os.path.dirname(Path(os.getcwd()))
sys.path.append(parent_dir)
import geomapi.utils.geometryutils as gmu
import geomapi.utils as ut
from geomapi.nodes import *

In [ ]:
#DATA
from data_loader_road import DataLoaderRoad
dataLoaderRoad=DataLoaderRoad(Path.cwd()/"testfiles"  )

from data_loader_parking import DataLoaderParking
dataLoaderParking=DataLoaderParking(Path.cwd()/"testfiles"  )

Creating Road DataLoader:
    loaded d:\geomapi\tests\testfiles\graphs\parking_resource_graph.ttl
    loaded 64 bimNodes from ifc file
    loaded d:\geomapi\tests\testfiles\graphs\road_ifc_graph.ttl
    loaded PointCloud with 700000 points.
    loaded d:\geomapi\tests\testfiles\pcd\lidar.e57
    loaded d:\geomapi\tests\testfiles\graphs\pcd_graph.ttl
    loaded TriangleMesh with 26002 points and 49999 triangles.
    loaded d:\geomapi\tests\testfiles\graphs\mesh_graph.ttl
    loaded d:\geomapi\tests\testfiles\graphs\road_img_graph.ttl
    loaded d:\geomapi\tests\testfiles\img\101_0367_0007.JPG
    loaded d:\geomapi\tests\testfiles\img\101_0367_0055.JPG
DataLoader succesfully loaded in 6.031972885131836 seconds!
Creating Parking DataLoader:
    loaded d:\geomapi\tests\testfiles\graphs\parking_resource_graph.ttl
    loaded PointCloud with 556485 points.
    loaded d:\geomapi\tests\testfiles\pcd\lidar.e57
    loaded d:\geomapi\tests\testfiles\pcd\parking.e57
    loaded d:\geomapi\tests\test

In [ ]:
# # imgNodes=[geomapi.imgNode(xmlPath=imageXmpPath1),imgNode(xmlPath=imageXmpPath1)]
# imgNode1=ImageNode(xmpPath=imageXmpPath1,getResource=True)
# imgNode2=ImageNode(xmpPath=imageXmpPath2,getResource=True)
# meshNode=MeshNode(path=meshPath,getResource=True)
# pcdNode=PointCloudNode(path=pcdPath,getResource=True)

In [ ]:
# #get list of files
# files=[str(path / 'IMG' / 'DJI_0085.xmp'),str(path / 'IMG' / 'IMG_8834.xmp')]
# print(files)
# #select every fifth files
# imgNodes=[]
# for f in files:     
#     n=ImageNode(name=ut.get_filename(f),  #! this took quite long from remote drive                            
#                                 xmpPath=f,getResource=True)
#     del n.resource
#     if getattr(n,'imageWidth',None) is not None:
#     #delete resource
    

#         #set focallength and cartesianTransform
#         n.focalLength35mm=n.focalLength35mm/36*n.imageWidth #! multiple definitions possible    
#         rotation= n.cartesianTransform[0:3,0:3].T #! RC uses column-based rotaton matrix
#         translation=n.cartesianTransform[0:3,3]
#         n.cartesianTransform=gmu.get_cartesian_transform(rotation=rotation,translation=translation)
        
#         imgNodes.append(n)
    
# number=len(imgNodes)
# imgNodes=[n for n in imgNodes if all(n.cartesianTransform[0:3,3] !=0)]
# joinedImages=gmu.join_geometries([gmu.generate_visual_cone_from_image(n.cartesianTransform, height =1).paint_uniform_color([1,0,0]) for n in imgNodes])
# print(f"{len(imgNodes)} / {number} Nodes created (remainder is not properly localised)!")

['D:\\geomapi\\tests\\testfiles\\IMG\\DJI_0085.xmp', 'D:\\geomapi\\tests\\testfiles\\IMG\\IMG_8834.xmp']
2 / 2 Nodes created (remainder is not properly localised)!


In [ ]:
node= MeshNode(subject=dataLoaderRoad.meshSubject,
                    path=dataLoaderRoad.meshPath,
                    graph=dataLoaderRoad.meshGraph,
                    getResource=True)
print(node.subject,dataLoaderRoad.meshSubject)
node.to_graph()
initialGraph=ut.get_subject_graph(dataLoaderRoad.meshGraph,subject=dataLoaderRoad.meshSubject)
print(len(node.graph),len(initialGraph))
box= dataLoaderRoad.mesh.get_axis_aligned_bounding_box()
min=box.get_min_bound()
print(node.cartesianBounds[0],min[0],delta=0.01)

AttributeError: 'DataLoaderRoad' object has no attribute 'meshSubject'

In [12]:
sourceMesh=gmu.mesh_to_trimesh(dataLoaderParking.mesh)
cutters=[gmu.mesh_to_trimesh(mesh) for mesh in dataLoaderParking.bimMeshes]

innerCrop=gmu.crop_mesh_by_convex_hull(source=sourceMesh, cutters=cutters, inside = True )
outerCrop=gmu.crop_mesh_by_convex_hull(source=sourceMesh, cutters=cutters[0], inside = False ) 

print(outerCrop)
# len(innerCrop)
# len(innerCrop[0].vertices)
# len(innerCrop[1].vertices)

[<trimesh.Trimesh(vertices.shape=(29271, 3), faces.shape=(53387, 3))>]


In [132]:
boxGt=dataLoaderRoad.mesh.get_axis_aligned_bounding_box()
boxGt.color=[0,1,0]
boxPointsGt=np.asarray(boxGt.get_box_points())
cartesianBounds=gmu.get_cartesian_bounds(dataLoaderRoad.mesh)
box=gmu.get_oriented_bounding_box(cartesianBounds)
box.color=[1,0,0]
boxPoints=np.asarray(box.get_box_points())


In [135]:
o3d.visualization.draw_geometries( [dataLoaderRoad.mesh,box])

In [7]:
ut.cartesianTransform_to_literal(imageCartesianTransform1)

'[[-8.13902571e-02]\n [ 6.83059476e-01]\n [-7.25813597e-01]\n [ 5.18276221e+01]\n [ 9.96648497e-01]\n [ 4.97790854e-02]\n [-6.49139139e-02]\n [ 6.10007435e+01]\n [-8.20972697e-03]\n [-7.28664391e-01]\n [-6.84821733e-01]\n [ 1.50408221e+01]\n [ 0.00000000e+00]\n [ 0.00000000e+00]\n [ 0.00000000e+00]\n [ 1.00000000e+00]]'

In [10]:
assert '-8.1390571e-02' in ut.cartesianTransform_to_literal(imageCartesianTransform1)

AssertionError: 